# Introduction to PyTorch

## Why PyTorch

1. Easy to use
2. Strong GPU support (fast)
3. Many algorithms are already implemented
4. Automatic differentiation
5. Similar to Numpy (easy to learn)

## PyTorch compared to NumPy

**一般 operations：**
![](Image/Image1.jpg)

---

**矩陣乘法：**
![](Image/Image2.jpg)

---

**Element-wise 乘法：**
![](Image/Image3.jpg)

---

**產生 0 矩陣以及 1 矩陣**
![](Image/Image4.jpg)

### Torch Tensor and NumPy Array conversion

Torch Tensor 轉換成 NumPy Array： tensor.numpy() 函數

In [6]:
# import
import torch

# define a torch tensor
t = torch.tensor([[1, 2, 3], [4, 5, 6]])

# convert tensor into array and print
print(t.numpy())

[[1 2 3]
 [4 5 6]]


NumPy Array 轉換成 Torch Tensor： torch.from_numpy(ARRAY) 函數

In [8]:
# import
import numpy as np

# define a numpy array
a = np.array([[1, 2, 3], [4, 5, 6]])
print(a)

# convert array into tensor
t_a = torch.from_numpy(a)
print(t_a)

[[1 2 3]
 [4 5 6]]
tensor([[1, 2, 3],
        [4, 5, 6]], dtype=torch.int32)


## PyTorch 重要的 operations 的函數

![](Image/Image5.jpg)

# Backpropagation by auto-differentiation

深度學習中最重要的數學就是計算微分，因為微分在幾何上代表了函數的斜率 (**Gradient**)，當斜率為 0 時就代表找到極值。因此，這裡要學習如何計算一個函數的偏微分。

![](Image/Image6.jpg)

上圖為一個函數的 computational graph。可知此函數 $f = q * z$，而 $q = x + y$。我們要計算 $f$ 在 $x$, $y$, $z$, $q$ 四個方向的微分值 (藍色的數字)。

1. $f$ 的微分值必為 1，因為對自己微分必為 1
2. $f$ 對 $q$ 的微分值為 -2。根據乘法定理，$$\frac{df}{dq}\ = \frac{d}{dq}\ (q*z) = \frac{dq}{dq}\ * z + q * \frac{dz}{dq}\ = z = -2 $$
3. $f$ 對 $z$ 的微分值為 2。根據乘法定理，$$\frac{df}{dz}\ = \frac{d}{dz}\ (q*z) = \frac{dq}{dz}\ * z + q * \frac{dz}{dz}\ = q = 2 $$
4. $f$ 對 $x$ 的微分值為 -2。根據乘法定理，$$\frac{df}{dx}\ = \frac{d}{dx}\ [(x+y)z] = \frac{d(x+y)}{dx}\ * z + (x+y) * \frac{dz}{dx}\ = z = -2 $$
5.  $f$ 對 $y$ 的微分值為 -2。根據乘法定理，$$\frac{df}{dy}\ = \frac{d}{dy}\ [(x+y)z] = \frac{d(x+y)}{dy}\ * z + (x+y) * \frac{dz}{dy}\ = z = -2 $$

## Implementation in PyTorch

上面的微分都可以透過 PyTorch 的 backward() 函數來實踐。例如：

In [21]:
# import 
import torch

# requires_grad 代表告知 PyTorch 我們要計算 gradients
x = torch.tensor(-3., requires_grad = True)
y = torch.tensor(5., requires_grad = True)
z = torch.tensor(-2., requires_grad = True)

# define the function
q = x + y
f = q * z

# 計算 gradients
f.backward()

In [22]:
# 印出結果
print("f 在 x 向量方向的 gradient 為 {}".format(str(x.grad)))
print("f 在 y 向量方向的 gradient 為 {}".format(str(y.grad)))
print("f 在 z 向量方向的 gradient 為 {}".format(str(z.grad)))

f 在 x 向量方向的 gradient 為 tensor(-2.)
f 在 y 向量方向的 gradient 為 tensor(-2.)
f 在 z 向量方向的 gradient 為 tensor(2.)


課堂練習

In [23]:
x = torch.rand(2, 2, requires_grad = True)
y = torch.rand(2, 2, requires_grad = True)
z = torch.rand(2, 2, requires_grad = True)

# 矩陣乘法
q = torch.matmul(x, y)

# element-wise 乘法
f = z * q

# 求 f 所有 element 的平均
mean_f = torch.mean(f)

# 計算 gradients
mean_f.backward()

In [24]:
# 印出結果
print("f 在 x 向量方向的 gradient 為 {}".format(str(x.grad)))
print("f 在 y 向量方向的 gradient 為 {}".format(str(y.grad)))
print("f 在 z 向量方向的 gradient 為 {}".format(str(z.grad)))

f 在 x 向量方向的 gradient 為 tensor([[0.0241, 0.1563],
        [0.0136, 0.1125]])
f 在 y 向量方向的 gradient 為 tensor([[0.0677, 0.2470],
        [0.0487, 0.1611]])
f 在 z 向量方向的 gradient 為 tensor([[0.2006, 0.1683],
        [0.1339, 0.1047]])


# Introduction to Neural Networks

一般的 regression models 或 classification models 在相對完整的資料 (不需要 extract 複雜的 features) 表現可以很好。然而，當遇到某些狀況，例如： image recognition 或 language processing 時，由於我們要從一堆數字中找出影像或音訊的 features，而傳統的模型無法找出這些藏在數字中的 features。因此，我們才需要 neural networks 來找出潛在的 features。例如：Convolutional Neural Networks 可以利用 kernels 來找出特定的圖案 patterns。

## Implementation in PyTorch (Linear Algebra approach)
假設現在 input layer 有 10 個 nodes，第一個 hidden layer 有 20 個 nodes，第二個 hidden layer 有 20 個 nodes，而 output layer 有 4 個 nodes，且 layers 都屬於 Dense layer，則：

In [29]:
# import
import torch

# 隨機產生 input layer
inputs_layer = torch.rand(10)

# 第一個 hidden layer 與 input layer 之間的 weights
weight1 = torch.rand(10, 20)

# 計算第一個 hidden layer 中各個 nodes 的值
hidden1 = torch.matmul(inputs_layer, weight1)

# 第二個 hidden layer 與 第一個 hidden layer 之間的 weights
weight2 = torch.rand(20, 20)

# 計算第二個 hidden layer 中各個 nodes 的值
hidden2 = torch.matmul(hidden1, weight2)

# output layer 與 第二個 hidden layer 之間的 weights
weight_out = torch.rand(20, 4)

# 計算 4 個輸出
outputs_layer = torch.matmul(hidden2, weight_out)

# 印出結果
print(outputs_layer)

tensor([317.2751, 299.5027, 231.7783, 256.8259])


上面的方法利用線性代數的矩陣乘法原理來計算 (forward propagation) 最後的輸出，是屬於較 low level 的方法，非常彈性，但建模過程麻煩且不直觀。 PyTorch 有提供較高階的方法 (跟 Tensorflow 的 keras.layers.Dense 類似)，可以較直覺的建立 neural networks 的模型。

PyTorch 中高階的模型是一種物件導向的寫法 (Object-oriented)。寫法如下：

In [30]:
# import
import torch 
import torch.nn as nn

# 建立模型的類別
class Net(nn.Module):
    # constructor 中我們會透過 specify weights 來定義模型中各個 layers 的 nodes 的數量 
    def __init__(self):
        super(Net, self).__init__()
        # fully connected layers (dense layers) 在 PyTorch 中稱為 nn.Linear(目前layer的nodes數, 下一層layer的nodes數)
        self.fc1 = nn.Linear(10, 20)
        self.fc2 = nn.Linear(20, 20)
        self.output = nn.Linear(20, 4)
        
    def forward(self, x):    # inputs 為 x
        # apply all weights to the inputs
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.output(x)
        return x

建立模型的物件，傳入 inputs，並且呼叫 forward method 來計算

In [31]:
inputs_layer2 = torch.rand(10)

# 建立模型物件
net = Net()

# 計算輸出結果
outputs_layer2 = net.forward(inputs_layer2)

# 印出結果
print(outputs_layer2)

tensor([ 0.2081,  0.0588, -0.0061, -0.1764], grad_fn=<AddBackward0>)
